# Data Load

In [ ]:
from tqdm import tqdm
import pickle

import numpy as np
import math
import random
import pandas as pd

import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data.dataset import random_split

In [ ]:
with open('data/train_dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)
print ("finish loading dataset")

In [ ]:
random.seed(0)
random.shuffle(dataset)

In [ ]:
def devide_dataset(dataset, start_ratio, end_ratio):
    train_dataset = dataset[:int(len(dataset)*start_ratio)] + dataset[int(len(dataset)*end_ratio):]
    valid_dataset = dataset[int(len(dataset)*start_ratio):int(len(dataset)*end_ratio)]
    return train_dataset, valid_dataset


---

# Network

In [ ]:
gpu_device = 0

EMBEDDING_SIZE = 10+25+1
ENCODING_SIZE = 32
SEQ_LENGTH = 15

batch_size = 512
num_epochs = 20
learning_rate = 1e-2
clipping = 0.15
criterion = nn.CrossEntropyLoss()

In [ ]:
def fit(model,train_loader,valid_loader,criterion,learning_rate,num_epochs,model_name):
    best_mrr = -9999
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = lr_scheduler.MultiStepLR(optimizer, [10,15], gamma=0.2)
    for epoch in range(num_epochs):
        model.train()
        for i, data in enumerate(train_loader):
            session = data[0].type(torch.FloatTensor).cuda(gpu_device)
            display = data[1].type(torch.FloatTensor).cuda(gpu_device)
            encoding = data[2].type(torch.FloatTensor).cuda(gpu_device)
            criteria = data[3].type(torch.FloatTensor).cuda(gpu_device)
            device = data[4].type(torch.FloatTensor).cuda(gpu_device)
            real_clicked_item = data[5].type(torch.LongTensor).cuda(gpu_device)
            
            output = model(session, display, encoding, criteria, device)
            loss1 = criterion(output[0], real_clicked_item)
            loss2 = criterion(output[1], real_clicked_item)
            loss = 0.10*loss1 + 1.00*loss2
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clipping)
            optimizer.step()
        
        
        real_all, pred_all = eval(model, valid_loader, model_name)
        now_mrr = calculate_mrr(pred_all, real_all)
        if best_mrr < now_mrr:
            best_mrr = now_mrr
            print ('epoch {} - mrr: {}'.format(epoch, best_mrr))
            file_name = 'best_valid_'+model_name+'.pth'
            torch.save(model.state_dict(), file_name)
        
        scheduler.step()

In [ ]:
def calculate_mrr(pred, real):
    reciprocal_rank = []
    for idx in range(len(pred)):
        reciprocal_rank.append(1/(np.where(np.argsort(pred[idx])[::-1]==(real[idx]))[0][0]+1))
    mrr = np.mean(reciprocal_rank)
    return mrr

In [ ]:
def eval(model,test_loader,model_name):
    model.eval()
    real_all = []
    pred_all = []

    for i, data in enumerate(test_loader):
        session = data[0].type(torch.FloatTensor).cuda(gpu_device)
        display = data[1].type(torch.FloatTensor).cuda(gpu_device)
        encoding = data[2].type(torch.FloatTensor).cuda(gpu_device)
        criteria = data[3].type(torch.FloatTensor).cuda(gpu_device)
        device = data[4].type(torch.FloatTensor).cuda(gpu_device)
        real_clicked_item = data[5].type(torch.LongTensor).cuda(gpu_device)
        real_all += real_clicked_item.cpu().detach().numpy().tolist()
        
        output = model(session, display, encoding, criteria, device)
        pred_all += output[1].cpu().detach().numpy().tolist()
        
    real_all = np.array(real_all)
    pred_all = np.array(pred_all)

    return real_all, pred_all

In [ ]:
idx_lst = list(range(39))
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=EMBEDDING_SIZE, hidden_size=32, num_layers=2, bidirectional=True, batch_first=True)
        self.lstm_fc = nn.Sequential(
            nn.Linear(64+12+3, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 25))
        
        self.conv1x1 = nn.Sequential(
            nn.Conv1d(1+ENCODING_SIZE+len(idx_lst), 64, kernel_size=1, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU())
        
        self.conv =  nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=3, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=3, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU())
        
        self.conv_fc = nn.Sequential(
            nn.Linear(256*15, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.8))
        
        self.final_fc = nn.Linear(512, 25)

    def forward(self, session, display, encoding, criteria, device):
        output, (hidden, cell) = self.lstm(session, None)
        output = output[:,SEQ_LENGTH-1,:]
        output = torch.cat([output, criteria, device], dim=1)
        session_output = self.lstm_fc(output)

        display = display[:,idx_lst,:]
        display = torch.cat([session_output.view(-1, 1, 25), encoding, display], dim=1)
        display_output = self.conv1x1(display)
        conv_output = self.conv(display_output).view(-1, 256*15)
        final_feature = self.conv_fc(conv_output)
        final_output = self.final_fc(final_feature)
        return session_output, final_output

In [ ]:
for i in range(5):
    start_ratio = i*0.2
    end_ratio = start_ratio+0.2
    train_dataset, valid_dataset = devide_dataset(dataset, start_ratio, end_ratio)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=6, pin_memory=True, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, num_workers=6, pin_memory=True, shuffle=False)

    net = Net().cuda(gpu_device)
    print (start_ratio)
    fit(net, train_loader, valid_loader, criterion, learning_rate, num_epochs, 'net{}'.format(i+1))

---

# Test

In [ ]:
with open('data/line_final.pickle', 'rb') as f:
    submissions = pickle.load(f)
with open('data/test_dataset.pickle', 'rb') as f:
    test_dataset = pickle.load(f)

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for i in range(len(submissions)):
    if len(submissions[i][4]) != 25:
        submissions[i][4] = submissions[i][4] + ['-9999']*(25-len(submissions[i][4]))

In [ ]:
def load_net(idx):
    net = Net().cuda(gpu_device)
    file_name = 'best_valid_net{}.pth'.format(idx)
    net.load_state_dict(torch.load(file_name))
    net.eval()
    return net

In [ ]:
ensemble_pred = np.zeros([len(test_dataset), 25])
for idx in range(1, 6):
    net = load_net(idx)
    pred_all = []
    for i, data in enumerate(test_loader):
        session = data[0].type(torch.FloatTensor).cuda(gpu_device)
        display = data[1].type(torch.FloatTensor).cuda(gpu_device)
        encoding = data[2].type(torch.FloatTensor).cuda(gpu_device)
        criteria = data[3].type(torch.FloatTensor).cuda(gpu_device)
        device = data[4].type(torch.FloatTensor).cuda(gpu_device)

        output = net(session, display, encoding, criteria, device)
        pred_all += list(F.softmax(output[1], dim=1).cpu().detach().numpy())
    ensemble_pred += np.array(pred_all)

In [ ]:
sorted_index = np.argsort(-ensemble_pred)

for i in range(len(sorted_index)):
    tmp_submission = submissions[i][4]
    sorted_submission = [tmp_submission[idx] for idx in sorted_index[i] if tmp_submission[idx]!='-9999']
    sorted_submission = ' '.join(sorted_submission)
    submissions[i].append(sorted_submission)

submission_df = pd.DataFrame(submissions, columns=['user_id', 'session_id', 'timestamp', 'step', 'impressions', 'item_recommendations'])
submission_df = submission_df[['user_id', 'session_id', 'timestamp', 'step', 'item_recommendations']]
submission_df.to_csv('final_submission.csv', index=False)

---